Import our essential Libraries


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Input, Activation
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.vgg16 import VGG16

Mount your google drive so that the dataset can be cloned to it from the git.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Some shell script to check if the data already exists, if not clone it from git.



In [ ]:
!if [ ! -d '/content/gdrive/MyDrive/pneumoniaDataset' ]; then mkdir '/content/gdrive/MyDrive/pneumoniaDataset'; fi


!if [ ! -d '/content/gdrive/MyDrive/pneumoniaDataset/.git' ]; then git clone "https://github.com/Amzo/xray_images" '/content/gdrive/MyDrive/pneumoniaDataset/'; fi

Load our data and generate additional augmented data due to the nature of the small data set. Since the data is already structured into train, test and validate folders, we don't need to split the data here.


In [ ]:
def getData(trainDir, testDir, valDir):
        class_names = os.listdir(trainDir)
        class_types = len(os.listdir(trainDir))

        print('Number of classes for Classification: ',class_types)
        print(f'The class names are {class_names[0]} and {class_names[1]}')
        print('--> Count of Train Images <--')

        for i in class_names:
                print(i + ':' + str(len(os.listdir(trainDir + "/" +i))))
        print('--> Count of Test Images <--')

        for i in class_names:
                print(i + ':' + str(len(os.listdir(testDir + '/' +i))))

        print('--> Count of Validation Images <---')
        for i in class_names:
                print(i + ':' + str(len(os.listdir(valDir + '/' +i))))

        train_datagen = ImageDataGenerator(
                rescale=1/255.0,
                rotation_range=7,
                width_shift_range=0.5,
                height_shift_range=0.45,
                shear_range=0.2,
                zoom_range=0.45,
                horizontal_flip=True
        )

        test_datagen = ImageDataGenerator(rescale=1./255)

        xTrainGen = train_datagen.flow_from_directory(
                trainDir,
                target_size=(224,224),
                shuffle=True,
                batch_size=24,
                class_mode='binary'
        )

        xTestGen = test_datagen.flow_from_directory(
                testDir,
                target_size=(224,224),
                batch_size=16,
                shuffle=True,
                class_mode='binary'
        )

        xValGen = train_datagen.flow_from_directory(
                valDir,
                target_size=(224,224),
                batch_size=32,
                class_mode='binary'
        )

        return xTrainGen, xTestGen, xValGen

In [ ]:
def myModel():
  print("Defaulting to basic CNN")
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224,224,3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.5))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.6))
  model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.7))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"])
  
  return model

Create an Exception model. This model needs a lot of resources and will not run on my system.

Using RMSprop optimiser and setting a learning rate to 0.0001. As our classification is either true or false, E.G they have something or they don't, use binary_crossentropy.


In [ ]:
def modelBuild(inputShape, modelType):
  model = Sequential()
  if (modelType == "xception"):
    print("Setting up xception model")
    xception = Xception(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(xception)

  elif (modelType == "resnet"):
    print("Setting up resnet50 model")
    resnet = ResNet50V2(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(resnet)

  elif (modelType == "vgg16"):
    print("Setting up vgg16 model")
    vgg = VGG16(include_top=False,
      weights= 'imagenet',
      input_shape=inputShape,
    )
    model.add(vgg)

  model.add(Flatten())
  model.add(Dense(512, activation="relu"))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"])

  return model


In [ ]:
def modelTrain(model, xTrain, xVal, batchSize):
  model.fit(xTrain,
    epochs=10,
    validation_data=xVal,
    verbose=1,
    batch_size=batchSize
  )

  return model


In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"


inputTrain = '/content/gdrive/MyDrive/pneumoniaDataset/train'
inputTest = '/content/gdrive/MyDrive/pneumoniaDataset/test'
inputValidate = '/content/gdrive/MyDrive/pneumoniaDataset/val'
imageSize = (224,224,3)

xTrain, xTest, xVal = getData(inputTrain, inputTest, inputValidate)


Number of classes for Classification:  2
The class names are NORMAL and PNEUMONIA
--> Count of Train Images <--
NORMAL:1305
PNEUMONIA:3851
--> Count of Test Images <--
NORMAL:234
PNEUMONIA:390
--> Count of Validation Images <---
NORMAL:44
PNEUMONIA:32
Found 5156 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 76 images belonging to 2 classes.


In [ ]:
xception = modelBuild(imageSize, 'xception')
vgg16 = modelBuild(imageSize, 'vgg16')
resnet = modelBuild(imageSize, 'resnet')
myCNN = myModel()



Setting up xception model
83689472/83683744 [==============================] - 1s 0us/step
Setting up vgg16 model
58892288/58889256 [==============================] - 0s 0us/step
Setting up resnet50 model
94674944/94668760 [==============================] - 1s 0us/step
Defaulting to basic CNN


In [ ]:
trainedVGG = modelTrain(vgg16, xTrain, xVal, 32)


Epoch 1/10
215/215 [==============================] - 1386s 6s/step - loss: 0.5978 - accuracy: 0.7490 - val_loss: 0.5420 - val_accuracy: 0.7763
Epoch 2/10
215/215 [==============================] - 130s 603ms/step - loss: 0.3395 - accuracy: 0.8497 - val_loss: 0.6662 - val_accuracy: 0.8553
Epoch 3/10
215/215 [==============================] - 130s 602ms/step - loss: 0.3196 - accuracy: 0.8650 - val_loss: 0.3225 - val_accuracy: 0.8816
Epoch 4/10
215/215 [==============================] - 129s 600ms/step - loss: 0.2999 - accuracy: 0.8911 - val_loss: 0.4017 - val_accuracy: 0.8684
Epoch 5/10
215/215 [==============================] - 129s 597ms/step - loss: 0.2799 - accuracy: 0.8870 - val_loss: 0.9169 - val_accuracy: 0.7237
Epoch 6/10
215/215 [==============================] - 129s 600ms/step - loss: 0.2765 - accuracy: 0.8849 - val_loss: 0.4457 - val_accuracy: 0.8158
Epoch 7/10
215/215 [==============================] - 129s 597ms/step - loss: 0.2858 - accuracy: 0.8996 - val_loss: 0.4848 - v

In [ ]:
trainedResnet = modelTrain(resnet, xTrain, xVal, 32)

Epoch 1/10
215/215 [==============================] - 137s 587ms/step - loss: 0.7759 - accuracy: 0.8545 - val_loss: 44.3392 - val_accuracy: 0.5000
Epoch 2/10
215/215 [==============================] - 123s 570ms/step - loss: 0.2020 - accuracy: 0.9263 - val_loss: 114.5923 - val_accuracy: 0.9079
Epoch 3/10
215/215 [==============================] - 122s 568ms/step - loss: 0.1593 - accuracy: 0.9502 - val_loss: 804.1523 - val_accuracy: 0.5000
Epoch 4/10
215/215 [==============================] - 123s 570ms/step - loss: 0.1691 - accuracy: 0.9485 - val_loss: 0.1552 - val_accuracy: 0.9737
Epoch 5/10
215/215 [==============================] - 122s 569ms/step - loss: 0.4682 - accuracy: 0.9574 - val_loss: 0.4239 - val_accuracy: 0.9211
Epoch 6/10
215/215 [==============================] - 123s 570ms/step - loss: 0.1442 - accuracy: 0.9541 - val_loss: 1.0283 - val_accuracy: 0.8947
Epoch 7/10
215/215 [==============================] - 123s 569ms/step - loss: 0.1523 - accuracy: 0.9515 - val_loss: 27.

In [ ]:
trainedCNN = modelTrain(myCNN, xTrain, xVal, 32)

Epoch 1/10
215/215 [==============================] - 112s 513ms/step - loss: 0.8370 - accuracy: 0.5899 - val_loss: 2.3719 - val_accuracy: 0.4211
Epoch 2/10
215/215 [==============================] - 110s 510ms/step - loss: 0.5795 - accuracy: 0.7193 - val_loss: 1.0787 - val_accuracy: 0.5395
Epoch 3/10
215/215 [==============================] - 110s 510ms/step - loss: 0.4418 - accuracy: 0.7965 - val_loss: 1.4338 - val_accuracy: 0.4342
Epoch 4/10
215/215 [==============================] - 109s 509ms/step - loss: 0.3764 - accuracy: 0.8308 - val_loss: 3.0499 - val_accuracy: 0.4474
Epoch 5/10
215/215 [==============================] - 110s 512ms/step - loss: 0.3529 - accuracy: 0.8500 - val_loss: 2.3699 - val_accuracy: 0.4342
Epoch 6/10
215/215 [==============================] - 110s 510ms/step - loss: 0.3216 - accuracy: 0.8645 - val_loss: 3.6166 - val_accuracy: 0.4211
Epoch 7/10
215/215 [==============================] - 109s 509ms/step - loss: 0.3312 - accuracy: 0.8606 - val_loss: 1.6837 -

In [ ]:
trainedXception = modelTrain(xception, xTrain, xVal, 32)

Epoch 1/10
215/215 [==============================] - 156s 676ms/step - loss: 0.6786 - accuracy: 0.8554 - val_loss: 4.5295 - val_accuracy: 0.6053
Epoch 2/10
215/215 [==============================] - 141s 654ms/step - loss: 0.1503 - accuracy: 0.9487 - val_loss: 4.6786 - val_accuracy: 0.7105
Epoch 3/10
215/215 [==============================] - 141s 655ms/step - loss: 0.1422 - accuracy: 0.9615 - val_loss: 1.7767 - val_accuracy: 0.8816
Epoch 4/10
215/215 [==============================] - 141s 655ms/step - loss: 0.1106 - accuracy: 0.9650 - val_loss: 4.1040 - val_accuracy: 0.8289
Epoch 5/10
215/215 [==============================] - 141s 653ms/step - loss: 0.1116 - accuracy: 0.9611 - val_loss: 0.6023 - val_accuracy: 0.9474
Epoch 6/10
215/215 [==============================] - 141s 656ms/step - loss: 0.1054 - accuracy: 0.9632 - val_loss: 1.5675 - val_accuracy: 0.8421
Epoch 7/10
215/215 [==============================] - 141s 654ms/step - loss: 0.0928 - accuracy: 0.9681 - val_loss: 13.2958 

In [ ]:
print("Running Xception model against test set")
ev = trainedXception.evaluate(xTest)
print("\n%s: %.f%%" % (trainedXception.metrics_names[1], ev[1]*100))
print("Running VGG16 model against test set")
ev = trainedVGG.evaluate(xTest)
print("\n%s: %.f%%" % (trainedVGG.metrics_names[1], ev[1]*100))
print("Running resnet model against test set")
ev = trainedResnet.evaluate(xTest)
print("\n%s: %.f%%" % (trainedResnet.metrics_names[1], ev[1]*100))
print("Running myCNN model against test set")
ev = trainedCNN.evaluate(xTest)
print("\n%s: %.f%%" % (trainedCNN.metrics_names[1], ev[1]*100))

Running Xception model against test set


NameError: ignored

In [ ]:
trainedXception.save('/content/drive/MyDrive/inception')
trainedVGG.save('/content/drive/MyDrive/VGG')
trainedResnet.save('/content/drive/MyDrive/Resnet')
trainedCNN.save('/content/drive/MyDrive/myCNN')

NameError: ignored